In [8]:
import pandas as pd
import datetime as dt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

movies_df = pd.read_csv('../../Resources/modified_movies.csv',low_memory=False)
movies_df.sort_values(by=['total_votes'], inplace=True, ascending=False)

In [35]:
movies_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'imdb_title_id', 'title',
       'original_title', 'year', 'date_published', 'genre', 'duration',
       'country', 'language', 'director', 'writer', 'production_company',
       'actors', 'description', 'avg_vote', 'votes', 'budget',
       'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics', 'id', 'encoded_genre',
       'writer_encoded', 'director_encoded', 'language_encoded',
       'weighted_average_vote', 'total_votes', 'mean_vote', 'median_vote',
       'votes_10', 'votes_9', 'votes_8', 'votes_7', 'votes_6', 'votes_5',
       'votes_4', 'votes_3', 'votes_2', 'votes_1', 'allgenders_0age_avg_vote',
       'allgenders_0age_votes', 'allgenders_18age_avg_vote',
       'allgenders_18age_votes', 'allgenders_30age_avg_vote',
       'allgenders_30age_votes', 'allgenders_45age_avg_vote',
       'allgenders_45age_votes', 'males_allages_avg_vote',
       'males_allages_votes', 'males_0age_avg_vote

In [2]:
sub_movies = movies_df[movies_df['language'].str.contains('English', case=False)] 
sub_movies.reset_index(drop=True, inplace=True)
print(sub_movies.shape)
print(sub_movies.head())

(34343, 78)
   Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1 imdb_title_id  \
0          12            12           24403     tt0111161   
1          48            48           38972     tt0468569   
2         118           118           45652     tt1375666   
3         103           103           27399     tt0137523   
4          53            53           24349     tt0110912   

                  title            original_title  year date_published  \
0  Le ali della libertà  The Shawshank Redemption  1994     1995-02-10   
1   Il cavaliere oscuro           The Dark Knight  2008     2008-07-23   
2             Inception                 Inception  2010     2010-09-24   
3            Fight Club                Fight Club  1999     1999-10-29   
4          Pulp Fiction              Pulp Fiction  1994     1994-10-28   

                               genre  duration  ... females_30age_avg_vote  \
0                          ['Drama']       142  ...                    9.2   
1       ['Action', '

In [55]:
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
sub_movies['description']=sub_movies['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(sub_movies['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

#tfidf.get_feature_names()[1000:1020]

<ipython-input-55-36d73f0a0ee0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_movies['description']=sub_movies['description'].fillna('')


(34343, 37103)

In [56]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape)

indices = pd.Series(sub_movies.index, index=sub_movies['original_title']).drop_duplicates()
print(indices.size)


In [24]:
def fuzzy_search(title:str, with_ratio =  False):
    list = []
    for word in title.split(" "):
        list.append(word.capitalize()[0]) 
    if 'T' not in list:
        list.append('T')
    
    print(list)
    s = sub_movies.loc[sub_movies['original_title'].str.startswith(tuple(list)), 'original_title']
    #print(sorted(s, key=lambda x : x[0]))
    Ratios = process.extract(title ,s)
    if with_ratio:
        return Ratios
    else:
        return [x[0] for x in Ratios]

In [29]:
fuzzy_search("yesterday, I went to the movies and watched the avengers", True)

['Y', 'I', 'W', 'T', 'T', 'M', 'A', 'W', 'T', 'A']


[('The Avengers', 90, 17),
 ('Yesterday', 90, 1688),
 ('Yesterday', 90, 31195),
 ('The Shawshank Redemption', 86, 0),
 ('The Dark Knight', 86, 1)]

In [12]:
import difflib
import math 
import array
from fuzzywuzzy import process

def similarity(word, pattern):
    return difflib.SequenceMatcher(a=word.lower(), b=pattern.lower()).ratio()

def fuzzy_search(title, unzip_list=False):
    threshold = 0.6
    output = []
    outputWeight = []
    for lookup in sub_movies['original_title']:
        s =similarity(title, lookup) 
        if s > threshold:
            m = sub_movies[sub_movies['original_title'] == lookup].original_title
            try:
                output.append(m.item())
                outputWeight.append(s*10000)
            except Exception:
                pass
            #print(sub_movies[sub_movies['original_title'] == lookup].original_title)

    zipped_lists = zip(outputWeight,output)
    sorted_zipped_lists = sorted(zipped_lists, reverse=True)
    sorted_list1 = [element for _, element in sorted_zipped_lists]

    if (unzip_list):
        sorted_list1 = list(zip(*sorted_zipped_lists))
    
    
    return sorted_list1

#fuzzy_search("Harry Potter and the ")

def fuzz_matcher(string):
    Ratios = process.extract(string,sub_movies['original_title'].values)
    #print(Ratios)
    #highest = process.extractOne(string,sub_movies['original_title'].values)
    #print(highest)
    return Ratios

    

In [778]:
fuzz_matcher("Sam")

[('Sam', 100),
 ('The Last Samurai', 90),
 ('I Am Sam', 90),
 ('Chung Hing sam lam', 90),
 ('Transamerica', 90)]

In [772]:
def find_movie_in_string(string:str, window = 3):
    i = 0
    
    possible_movies_list = []
    words = string.split(" ")
    print(words)
    if len( words) >= 3:
        for i in range(0, (len( words) - window) + 1):
            print(" ".join(words[i:i+window]))
            w = fuzz_matcher(" ".join(words[i:i+window]))
            for j in range(10):
                if(w != [] and j < len(w)):
                    possible_movies_list.append((w[j][1], w[j][0] ))
                else:
                    break
    else:
        w = fuzz_matcher(string)
        for j in range(10):
            if(w != [] and j < len(w)):
                possible_movies_list.append((w[j][1], w[j][0] ))
            else:
                break
    

    possible_movies_list.sort( key=lambda x : x[0], reverse=True)

    print(possible_movies_list)
    for i in range(5):
        if(possible_movies_list != [] and i < len(possible_movies_list)):
            if string.lower().find(possible_movies_list[i][1].lower()) > -1:
                print("found your movie: ",possible_movies_list[i][1] )
                return possible_movies_list[i][1]
    
    
    if possible_movies_list != []:
        print("did you mean ", possible_movies_list[0][1], " ?")
        return possible_movies_list[0][1]
    else:
        return None

In [777]:
string = "avengers end game"
#print(fuzzy_search(string))
print(fuzz_matcher(string))
#find_movie_in_string(string)

[('Avengers: Endgame', 94), ('End Game', 90), ("Pirates of the Caribbean: At World's End", 86), ('Sherlock Holmes: A Game of Shadows', 86), ('The Game', 86)]


In [58]:
def recommendation(title,cos=cosine_sim):
    if title in indices.keys():
        idx=indices[title]
    elif fuzzy_search(title)[0] in indices.keys():
        idx=indices[fuzzy_search(title)[0]]
        print("did you mean" ,fuzzy_search(title)[0], "?")

    idx = idx[0] if(isinstance(idx, list) or (not np.isscalar(idx)) ) else idx
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:len(sim_scores)]
    #print(sim_scores)
    movie_indices = [[i[0], i[1],  sub_movies['original_title'].iloc[i[0]]] for i in sim_scores]

    return movie_indices 
    #return sub_movies['original_title'].iloc[movie_indices]

In [292]:
recommendation("The Lord of the Rings: The Return of the King")[1:11]

[[11, 0.20873295626329375, 'The Lord of the Rings: The Two Towers'],
 [7, 0.15278678771685827, 'The Lord of the Rings: The Fellowship of the Ring'],
 [27044, 0.1325964686774591, '30 Years to Life'],
 [28030, 0.12889357993578718, 'Desperate Search'],
 [19897, 0.1213977318715607, 'Bullet for a Badman'],
 [119, 0.11991560699822923, 'X-Men: Days of Future Past'],
 [1318, 0.11832718467807758, 'Conan the Barbarian'],
 [17709, 0.11484332120254412, 'Popcorn'],
 [31703, 0.11352555987110945, 'Men of Means'],
 [145, 0.11171706196642081, 'The Hobbit: The Desolation of Smaug']]

In [477]:
def getMovieName(title:str):
    title = title.strip()

    if title in indices.keys():
        return True, sub_movies[sub_movies['original_title'] == title].iloc[[0]]
    elif fuzzy_search(title)[0] in indices.keys():
        print("did you mean" ,fuzzy_search(title)[0], "?")
        return False, sub_movies[sub_movies['original_title'] == fuzzy_search(title)[0]].iloc[[0]]


    

In [479]:
getMovieName(" Fight Club")[1]

,Unnamed: 0,Unnamed: 0.1,imdb_title_id,title,original_title,year,date_published,genre,duration,country,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
3,103,27399,tt0137523,Fight Club,Fight Club,1999,1999-10-29,['Drama'],139,"USA, Germany",...,8.7,115892.0,8.1,20944.0,8.3,882.0,8.7,263329.0,8.8,727678.0


In [547]:
from scipy import spatial
from pandas import DataFrame
import numpy as np

def similarityFactor(movie1: DataFrame, movie2 : DataFrame, script_similarity, age=0, gender=0):
    similarity = 0

    value1 = np.array(movie1['director_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
    value2 = np.array(movie2['director_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
    padding = abs(value1.shape[0] - value2.shape[0])
    value1 = np.pad(value1, ( padding ,0), 'constant') if value1.shape[0] < value2.shape[0] else value1
    value2 = np.pad(value2, ( padding ,0), 'constant') if value1.shape[0] > value2.shape[0] else value2
    
    similarity_factor = 0.1 *difflib.SequenceMatcher(a= value1, b=value2).ratio()
    

    value1 = np.array(movie1['writer_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
    value2 = np.array(movie2['writer_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
    padding = abs(value1.shape[0] - value2.shape[0])
    value1 = np.pad(value1, ( padding ,0), 'constant') if value1.shape[0] < value2.shape[0] else value1
    value2 = np.pad(value2, ( padding ,0), 'constant') if value1.shape[0] > value2.shape[0] else value2
    
    similarity_factor += 0.1 * difflib.SequenceMatcher(a= value1, b=value2).ratio()

    value1 = np.array(movie1['encoded_genre'].values[0].strip("[").strip("]").split(), dtype=int)
    value2 = np.array(movie2['encoded_genre'].values[0].strip("[").strip("]").split(), dtype=int)
    padding = abs(value1.shape[0] - value2.shape[0])
    value1 = np.pad(value1, ( padding ,0), 'constant') if value1.shape[0] < value2.shape[0] else value1
    value2 = np.pad(value2, ( padding ,0), 'constant') if value1.shape[0] > value2.shape[0] else value2
    
    similarity_factor += 0.3 * difflib.SequenceMatcher(a= value1, b=value2).ratio()
    
    r = getMoveRating(movie2, age, gender)
    similarity_factor += 0.3 * (r/10)

    
    
    similarity_factor += 0.2 * script_similarity

    
    return similarity_factor , r

    

In [531]:
MALE = 1
FEMALE = 2
NONBINARY = 0

def getMoveRating(movie : DataFrame , age:int, gender:int):
    age_group = [0, 18, 30, 45]
    age_group = [abs(age_group[i]-age) for i in range(len(age_group))]
    group = age_group.index(min(age_group))
    attribute = ""
    attribute2 = ""

    if gender == 0 and age == 0:
        attribute = "avg_vote"
        attribute2 = "avg_vote"
    else:
        if gender == 0:
            attribute += "allgenders_"
            attribute2 += "allgenders_"
            
        elif gender == 1:
            attribute += "males_"
            attribute2 += "males_"
        else:
            attribute += "females_"
            attribute2 += "females_"

        if age == 0:
            attribute += "allages_avg_vote"
            attribute2 += "allages_votes"
        elif group == 0: # group 0
            attribute += "0age_avg_vote"
            attribute2 += "0age_votes"
        elif group == 1: # group 18
            attribute += "18age_avg_vote"
            attribute2 += "18age_votes"
        elif group == 1: # group 30
            attribute += "30age_avg_vote"
            attribute2 += "30age_votes"
        else: #group 45
            attribute += "45age_avg_vote"
            attribute2 += "45age_votes"
    

    avg = movie[attribute].values[0]
    num_voters = movie[attribute2].values[0]

    # rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C 
    # R = average for the movie (mean) = (Rating)
    # v = number of votes for the movie = (votes)
    # m = minimum votes required to be listed in the Top 250 (currently 3000)
    # C = the mean vote across the whole report (currently 6.9)
    M = 3000
    C = 6
    rating = ( num_voters / (num_voters + M )) * avg + (M / (num_voters + M)) * C

    #print(rating)


    return rating

In [462]:
getMoveRating( 1, FEMALE)

females_0age_avg_vote


In [434]:

import math
def recommend(movie):
    recommend_score = 0
    aux = 0
    movies_list = recommendation(movie['original_title'].item())
    movies_list = movies_list[1:math.floor(len(movies_list)*0.1)]
    print(math.floor(len(movies_list)*0.1))
    for m in movies_list: 
        found, name = getMovieName(m[2])
        aux = similarityFactor(movie,name, m[1] )
        if aux > recommend_score : 
            recommend_score = aux
            movie_to_recommend = name

    return recommend_score, movie_to_recommend

In [10]:
import numpy as np
manager = MoviesManager()

<ipython-input-5-903737ee15a6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.sub_movies['description']=self.sub_movies['description'].fillna('')


In [17]:
genres_loaded = np.load("../../Resources/genre.npy")
def fuzzy_find_genre( genre , with_ratio = False):        
    Ratios = process.extract(genre,genres_loaded)
    if with_ratio:
        return Ratios
    else:
        return [x[0] for x in Ratios]

In [39]:
def get_gender(string: str) -> int:

    non_binary = ['non-binary', 'other', 'gender fluid' ]
    male = ['male', 'Boy', 'man', 'guy' ]
    female = ['female', 'women', 'gal', 'girl', 'lady']
    
    ratios_nb = process.extract(string, non_binary)[0][1]
    ratios_m = process.extract(string, male)[0][1]
    ratios_f = process.extract(string, female)[0][1]
    
    ratios = [ratios_nb, ratios_m, ratios_f]

    max_value = max(ratios)

    if max_value < 80:
        return ratios.index(max_value), False

    return ratios.index(max_value), True
    


import random

def get_movie_sentence(movie):
    
    arr_sentences = [ movie + " is really good",  
                    movie + " is really good",
                    movie + " is very interesting",
                    movie + " is such a good movie",
                    "Good choice! I also like " + movie] 
    n = random.randint(0,len(arr_sentences) - 1)
    return arr_sentences[n]

In [62]:
get_gender("yes I am a boy")
#get_movie_sentence("The Avengers")

(1, True)

In [19]:
from youtubesearchpython import VideosSearch

import json


https://www.youtube.com/watch?v=eOrNdBpGMv8


In [5]:
from pandas import DataFrame
import pandas as pd

class MoviesManager:
    def __init__(self):
        movies_df = pd.read_csv('../../Resources/modified_movies.csv',low_memory=False)
        movies_df.sort_values(by=['total_votes'], inplace=True, ascending=False)
        
        self.sub_movies = movies_df[movies_df['language'].str.contains('English', case=False)] 
        self.sub_movies.reset_index(drop=True, inplace=True)
        self.genres_loaded = np.load("../../Resources/genre.npy")
        self.init_TFIDF()


    def init_TFIDF(self):
        tfidf = TfidfVectorizer(stop_words='english')
        #Replace NaN with an empty string
        self.sub_movies['description']=self.sub_movies['description'].fillna('')
        #Construct the required TF-IDF matrix by fitting and transforming the data
        tfidf_matrix = tfidf.fit_transform(self.sub_movies['description'])
        self.cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
        self.indices = pd.Series(self.sub_movies.index, index=self.sub_movies['original_title']).drop_duplicates()

    def similarity(self, word, pattern):
        return difflib.SequenceMatcher(a=word.lower(), b=pattern.lower()).ratio()
    '''
    def fuzzy_search(self, title):
        threshold = 0.6
        output = []
        outputWeight = []
        for lookup in self.sub_movies['original_title']:
            s =self.similarity(title, lookup) 
            if s > threshold:
                m = self.sub_movies[self.sub_movies['original_title'] == lookup].original_title
                try:
                    output.append(m.item())
                    outputWeight.append(s*10000)
                except Exception:
                    pass
                #print(sub_movies[sub_movies['original_title'] == lookup].original_title)

        zipped_lists = zip(outputWeight,output)
        sorted_zipped_lists = sorted(zipped_lists, reverse=True)
        sorted_list1 = [element for _, element in sorted_zipped_lists]

        return sorted_list1
    '''
    def fuzzy_search(self, title, with_ratio =  False):
        
        Ratios = process.extract(title ,self.sub_movies['original_title'].values)
        if with_ratio:
            return Ratios
        else:
            return [x[0] for x in Ratios]
    
    def fuzzy_find_genre(self, genre):        
        highest = process.extractOne(genre,self.genres_loaded)
        return highest[0]

    def find_similar_based_on_plot(self, title):
        
        if title in self.indices.keys():
            idx=self.indices[title]
        else:
            fuzz = self.fuzzy_search(title)[0]
            if fuzz in self.indices.keys():
                idx=self.indices[fuzz]
        
        idx = idx[0] if(isinstance(idx, list) or (not np.isscalar(idx)) ) else idx
        sim_scores = list(enumerate(self.cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:len(sim_scores)]
        #print(sim_scores)
        movie_indices = [[i[0], i[1],  self.sub_movies['original_title'].iloc[i[0]]] for i in sim_scores]

        return movie_indices 

    def getMovieName(self, title):

        if title in self.indices.keys():
            return True, self.sub_movies[self.sub_movies['original_title'] == title].iloc[[0]]
        else:
            t = self.fuzzy_search(title, True)
            if (t[0][1] > 90):
                print("assuming ", t[0][0], "")
                return True, self.sub_movies[self.sub_movies['original_title'] == t[0][0] ].iloc[[0]]
            else:
                value = input("did you mean ", t[0][0], " ? (y/n)")
                i = 0
                while(value != "y" and i < len(t)):
                    if(value == "n"):
                        i += 1
                        value = input("then did you mean ", t[i][0], " ? (y/n)")
                    else:
                        print("please enter y or n")

                return False, self.sub_movies[self.sub_movies['original_title'] == t[0][0] ].iloc[[0]]


    def getMoveRating(self, movie : DataFrame , age:int, gender:int):
        age_group = [0, 18, 30, 45]
        age_group = [abs(age_group[i]-age) for i in range(len(age_group))]
        group = age_group.index(min(age_group))
        attribute = ""
        attribute2 = ""

        if gender == 0 and age == 0:
            attribute = "avg_vote"
            attribute2 = "avg_vote"
        else:
            if gender == 0:
                attribute += "allgenders_"
                attribute2 += "allgenders_"
                
            elif gender == 1:
                attribute += "males_"
                attribute2 += "males_"
            else:
                attribute += "females_"
                attribute2 += "females_"

            if age == 0:
                attribute += "allages_avg_vote"
                attribute2 += "allages_votes"
            elif group == 0: # group 0
                attribute += "0age_avg_vote"
                attribute2 += "0age_votes"
            elif group == 1: # group 18
                attribute += "18age_avg_vote"
                attribute2 += "18age_votes"
            elif group == 1: # group 30
                attribute += "30age_avg_vote"
                attribute2 += "30age_votes"
            else: #group 45
                attribute += "45age_avg_vote"
                attribute2 += "45age_votes"
        

        avg = movie[attribute].values[0]
        num_voters = movie[attribute2].values[0]

        # rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C 
        # R = average for the movie (mean) = (Rating)
        # v = number of votes for the movie = (votes)
        # m = minimum votes required to be listed in the Top 250 (currently 3000)
        # C = the mean vote across the whole report (currently 6.9)
        M = 3000
        C = 6
        rating = ( num_voters / (num_voters + M )) * avg + (M / (num_voters + M)) * C

        return rating
    
    def similarityFactor(self, movie1: DataFrame, movie2 : DataFrame, script_similarity, age=0, gender=0):
        similarity = 0

        value1 = np.array(movie1['director_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
        value2 = np.array(movie2['director_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
        padding = abs(value1.shape[0] - value2.shape[0])
        value1 = np.pad(value1, ( padding ,0), 'constant') if value1.shape[0] < value2.shape[0] else value1
        value2 = np.pad(value2, ( padding ,0), 'constant') if value1.shape[0] > value2.shape[0] else value2
        
        similarity_factor = 0.05 *difflib.SequenceMatcher(a= value1, b=value2).ratio()
        

        value1 = np.array(movie1['writer_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
        value2 = np.array(movie2['writer_encoded'].values[0].strip("[").strip("]").split(), dtype=int)
        padding = abs(value1.shape[0] - value2.shape[0])
        value1 = np.pad(value1, ( padding ,0), 'constant') if value1.shape[0] < value2.shape[0] else value1
        value2 = np.pad(value2, ( padding ,0), 'constant') if value1.shape[0] > value2.shape[0] else value2
        
        similarity_factor += 0.05 * difflib.SequenceMatcher(a= value1, b=value2).ratio()

        try:
            if (abs(int(movie1['year'].values[0]) - int(movie2['year'].values[0])) != 0 ):
                similarity_factor += 0.03 * (1 /(abs(int(movie1['year'].values[0]) - int(movie2['year'].values[0]))))
            else: 
                similarity_factor += 0.03
        except ValueError: 
            similarity_factor += 0.01
        
        value1 = np.array(movie1['encoded_genre'].values[0].strip("[").strip("]").split(), dtype=int)
        value2 = np.array(movie2['encoded_genre'].values[0].strip("[").strip("]").split(), dtype=int)
        padding = abs(value1.shape[0] - value2.shape[0])
        value1 = np.pad(value1, ( padding ,0), 'constant') if value1.shape[0] < value2.shape[0] else value1
        value2 = np.pad(value2, ( padding ,0), 'constant') if value1.shape[0] > value2.shape[0] else value2
        
        similarity_factor += 0.36 * difflib.SequenceMatcher(a= value1, b=value2).ratio()
        
        r = self.getMoveRating(movie2, age, gender)
        similarity_factor += 0.31 * (r/10)

        
        
        similarity_factor += 0.2 * script_similarity

        
        return similarity_factor , r

        
    def recommend_from_user_list(self, user_movie_list: DataFrame, user_hate_movie_list:DataFrame, user_not_care_movie_list:DataFrame, age = 0, gender = 0, genre=None):
        recommend_score = 0
        aux = 0
        the_list = []
        genre = self.fuzzy_find_genre(genre)
        movie_to_recommend = None
        portion = 0.05/len(user_movie_list)
        for index, movie in user_movie_list.iterrows():
            movies_list = self.find_similar_based_on_plot(movie['original_title'])
            movies_list = movies_list[1:math.floor(len(movies_list)*portion)]
            the_list.extend(movies_list)
        the_list = [list(x) for x in set(tuple(x) for x in the_list)]
        looked_at = []
        the_list = sorted(the_list, key=lambda x: x[1], reverse=True)
        print(len(the_list))
        for m in the_list: 
            if m not in looked_at:
                looked_at.append(m)
                found, name = self.getMovieName(m[2])
                
                #print(name.genre, " : ", genre , "=>",  (genre not in name.genre.values[0]))
                if (genre is not None and genre not in name.genre.values[0]):
                    continue
                if (user_movie_list is not None and  name.original_title.isin(user_movie_list.original_title).astype(bool).all()):
                    continue
                if (user_hate_movie_list is not None and name.original_title.isin(user_hate_movie_list.original_title).astype(bool).all()):
                    continue
                if (user_not_care_movie_list is not None and name.original_title.isin(user_not_care_movie_list.original_title).astype(bool).all()):
                    continue
                aux = 0
                aux_hate = 0
                if user_movie_list is not None:
                    for index in range(len(user_movie_list)):
                        user_m = user_movie_list.iloc[[index]]
                        s, r = self.similarityFactor(user_m ,name, m[1], age, gender )
                        aux += s
                if user_hate_movie_list is not None:
                    for index in range(len(user_hate_movie_list)):
                        user_m = user_hate_movie_list.iloc[[index]]
                        s, r = self.similarityFactor(user_m ,name, m[1], age, gender )
                        aux_hate += s
                aux = aux - aux_hate
                aux /= len(user_movie_list)
                if aux > recommend_score : 
                    recommend_score = aux
                    movie_to_recommend = name
                    print(r)
                    
            
        return recommend_score, movie_to_recommend
    

In [794]:
from pandas import DataFrame
import pandas as pd
from colors import blue, red, yellow, green

class User:
    LIKE = 1
    DISLIKE = 2
    DONT_CARE = 3

    MALE = 1
    FEMALE = 2
    NONBINARY = 0
    
    def __init__(self):
        self.user_movie_list : DataFrame = None
        self.user_hate_movie_list : DataFrame = None
        self.user_not_care_movie_list : DataFrame = None
    
    def addMovie(self, movie:DataFrame, preference : int):
        if preference == self.LIKE: 
            if self.user_movie_list is None:
                self.user_movie_list = pd.concat([movie])
            else:
                self.user_movie_list = self.user_movie_list.append([movie])

        elif preference == self.DISLIKE:
            if self.user_hate_movie_list is None:
                self.user_hate_movie_list = pd.concat([movie])
            else:
                self.user_hate_movie_list = self.user_hate_movie_list.append([movie])

        else:
            if self.user_not_care_movie_list is None:
                self.user_not_care_movie_list = pd.concat([movie])
            else:
                self.user_not_care_movie_list = self.user_not_care_movie_list.append([movie])

    def print(self):
        print(green("like:"))
        if self.user_movie_list is not None:
            for i, m in self.user_movie_list.iterrows():
                print(m.original_title)
        print(green("dislike:"))
        if self.user_hate_movie_list is not None:
            for i, m in self.user_hate_movie_list.iterrows():
                print(m.original_title)
        print(green("not care:"))
        if self.user_not_care_movie_list is not None:
            for i, m in self.user_not_care_movie_list.iterrows():
                print(m.original_title)

    def getLikeList(self) -> DataFrame:
        return self.user_movie_list

    def getDislikeList(self) -> DataFrame:
        return self.user_hate_movie_list

    def getNotCareList(self) -> DataFrame:
        return self.user_not_care_movie_list 

In [811]:
manager = MoviesManager()

<ipython-input-810-c078cdd209d7>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.sub_movies['description']=self.sub_movies['description'].fillna('')


In [812]:
user = User()
found, entry = manager.getMovieName("Love, Simon")
user.addMovie(entry, 1)
while True:
    genre = input(('what genre of movies you feel like watching today:'))
    movie = input(('Please enter the name of a movie (enter "exit" to leave / enter "recommend" to recommend a movie):'))
    if (movie == "recommend"):
        score, result = manager.recommend_from_user_list(user.getLikeList(),user.getDislikeList(),user.getNotCareList(), age=20, gender=0, genre=genre)
        print(("similarity factor: "), (score))
        print( ("movie name: "), red(result['original_title'].item()))

        like=int(input(('do you like this movie (1. yes / 2. no / 3. don\'t care about it):')))
        user.addMovie(result, like)
    else:
        found, entry = manager.getMovieName(movie)
        like=int(input(('do you like this movie (1. yes / 2. no / 3. don\'t care about it):')))
        user.addMovie(entry, like)
    


1716
3524    ['Comedy', 'Romance']
Name: genre, dtype: object  :  Drama => True
31989    ['Western']
Name: genre, dtype: object  :  Drama => True
18786    ['Crime', 'Drama', 'Mystery']
Name: genre, dtype: object  :  Drama => False
5.999601593625498
2655    ['Action', 'Adventure', 'Romance']
Name: genre, dtype: object  :  Drama => True
3495    ['Comedy', 'Family']
Name: genre, dtype: object  :  Drama => True
8107    ['Comedy', 'Drama']
Name: genre, dtype: object  :  Drama => False
6.10036911719471
25164    ['Drama', 'Romance']
Name: genre, dtype: object  :  Drama => False
32635    ['Film-Noir', 'Mystery', 'Thriller']
Name: genre, dtype: object  :  Drama => True
19813    ['Comedy', 'Music', 'Romance']
Name: genre, dtype: object  :  Drama => True
29581    ['Comedy', 'Drama']
Name: genre, dtype: object  :  Drama => False
34071    ['Comedy', 'Drama']
Name: genre, dtype: object  :  Drama => False
19219    ['Crime', 'Drama', 'Mystery']
Name: genre, dtype: object  :  Drama => False
32611    ['

KeyboardInterrupt: 

In [490]:
sub_movies.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'imdb_title_id', 'title',
       'original_title', 'year', 'date_published', 'genre', 'duration',
       'country', 'language', 'director', 'writer', 'production_company',
       'actors', 'description', 'avg_vote', 'votes', 'budget',
       'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics', 'id', 'encoded_genre',
       'writer_encoded', 'director_encoded', 'language_encoded',
       'weighted_average_vote', 'total_votes', 'mean_vote', 'median_vote',
       'votes_10', 'votes_9', 'votes_8', 'votes_7', 'votes_6', 'votes_5',
       'votes_4', 'votes_3', 'votes_2', 'votes_1', 'allgenders_0age_avg_vote',
       'allgenders_0age_votes', 'allgenders_18age_avg_vote',
       'allgenders_18age_votes', 'allgenders_30age_avg_vote',
       'allgenders_30age_votes', 'allgenders_45age_avg_vote',
       'allgenders_45age_votes', 'males_allages_avg_vote',
       'males_allages_votes', 'males_0age_avg_vote

In [594]:
user_movie_list = pd.concat([
                getMovieName("Madagascar")[1],
                getMovieName("Madagascar: Escape 2 Africa")[1],
                getMovieName(" Madagascar 3: Europe's Most Wanted")[1],
                getMovieName("A Bug's Life")[1],
                getMovieName("Antz")[1],
                getMovieName(" Shrek")[1],
                ]
                )
user_hate_movie_list = pd.concat([
                
                getMovieName("Zootopia")[1],
                getMovieName("Tangled")[1],
                ]
                )

user_not_care_movie_list = pd.concat([
                getMovieName("Adventure Scouts")[1],
                getMovieName("Pitch Perfect")[1],
                ]
                )

score, result = recommend_from_user_list(user_movie_list,user_hate_movie_list,user_not_care_movie_list, age=30, gender=MALE)
print("similarity factor: ", score)
print( "movie name: ", result['original_title'].item() )
print( "statistics :" )
print()
print("avg vote: ", result['weighted_average_vote'].item(),  " , number of voters: ", result['total_votes'].item() )

print()
print("allgenders_0age_votes: ", result['allgenders_0age_avg_vote'].item(), " , number of voters: ", result['allgenders_0age_votes'].item() )
print("allgenders_18age_avg_vote: ", result['allgenders_18age_avg_vote'].item(), " , number of voters: ", result['allgenders_18age_votes'].item() )
print("allgenders_30age_avg_vote: ", result['allgenders_30age_avg_vote'].item(), " , number of voters: ", result['allgenders_30age_votes'].item() )
print("allgenders_45age_avg_vote: ", result['allgenders_45age_avg_vote'].item(), " , number of voters: ", result['allgenders_45age_votes'].item() )

print()
print("females_0age_votes: ", result['females_0age_avg_vote'].item(), " , number of voters: ", result['females_0age_votes'].item() )
print("females_18age_avg_vote: ", result['females_18age_avg_vote'].item(), " , number of voters: ", result['females_18age_votes'].item() )
print("females_30age_avg_vote: ", result['females_30age_avg_vote'].item(), " , number of voters: ", result['females_30age_votes'].item() )
print("females_45age_avg_vote: ", result['females_45age_avg_vote'].item(), " , number of voters: ", result['females_45age_votes'].item() )

print()
print("males_0age_votes: ", result['males_0age_avg_vote'].item(), " , number of voters: ", result['males_0age_votes'].item() )
print("males_18age_avg_vote: ", result['males_18age_avg_vote'].item(), " , number of voters: ", result['males_18age_votes'].item() )
print("males_30age_avg_vote: ", result['males_30age_avg_vote'].item(), " , number of voters: ", result['males_30age_votes'].item() )
print("males_45age_avg_vote: ", result['males_45age_avg_vote'].item(), " , number of voters: ", result['males_45age_votes'].item() )

3426
5.9758931445123915
5.962068965517241
5.9978160158835205
5.72864706956363
6.930177027583367
similarity factor:  0.4529658870204784
movie name:  Rango
statistics :

avg vote:  7.2  , number of voters:  231174

allgenders_0age_votes:  7.5  , number of voters:  247.0
allgenders_18age_avg_vote:  7.3  , number of voters:  59046.0
allgenders_30age_avg_vote:  7.2  , number of voters:  103846.0
allgenders_45age_avg_vote:  7.1  , number of voters:  19635.0

females_0age_votes:  7.4  , number of voters:  43.0
females_18age_avg_vote:  7.0  , number of voters:  13274.0
females_30age_avg_vote:  6.9  , number of voters:  17609.0
females_45age_avg_vote:  6.9  , number of voters:  2881.0

males_0age_votes:  7.5  , number of voters:  185.0
males_18age_avg_vote:  7.4  , number of voters:  44927.0
males_30age_avg_vote:  7.2  , number of voters:  85028.0
males_45age_avg_vote:  7.1  , number of voters:  16432.0


In [583]:
movie=input('Please enter the movie name:')

found, entry = getMovieName(movie)
score, result = recommend(entry)
print("similarity factor: ", score)
print( "movie name: ", result['original_title'].item() )



did you mean Madagascar ?
343


TypeError: '>' not supported between instances of 'tuple' and 'int'

In [ ]:
    movie = "Mamma Mia"
    arr_sentences[ movie + "is really good",  
                    movie + " is really good",
                    movie + " is very interesting",
                    movie + " is such a good movie",
                    "Good choice! I also like " + movie] 
    n = random.randint(0,len(arr_sentences))